In [1]:
from utils import Logger,adjust_learning_rate,CreateLogger,create_cosine_learning_schedule,encode_test_label,set_seed
from model import Resnet_with_uncertainty#不确定度
from dependency import *
from torch import optim
from torchcontrib.optim import SWA
from dataloader import generate_dataloader
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error # 均方误差

In [2]:
def train(net, train_dataloader, model_name):
    net.set_mode("train")
    train_loss = 0
    train_dia_acc = 0
    train_sps_acc = 0
    for index,(clinic_image,derm_image,meta_data,label) in enumerate(train_dataloader):
        opt.zero_grad()
        
        clinic_image = clinic_image.cuda()
        derm_image = derm_image.cuda()
        #meta_data = meta_data.cuda()
        
        #Diagnostic label
        diagnosis_label = label[0].long().cuda()
        #Seven-Point Checklist labels
        pn_label = label[1].long().cuda()
        str_label = label[2].long().cuda()
        pig_label = label[3].long().cuda()
        rs_label = label[4].long().cuda()
        dag_label = label[5].long().cuda()
        bwv_label = label[6].long().cuda()
        vs_label = label[7].long().cuda()
        #uncertainty
        
        [   (logit_diagnosis_clic,logit_pn_clic,logit_str_clic,logit_pig_clic,logit_rs_clic,logit_dag_clic,logit_bwv_clic,logit_vs_clic,logit_uncertainty_clic),
            (logit_diagnosis_derm,logit_pn_derm,logit_str_derm,logit_pig_derm,logit_rs_derm,logit_dag_derm,logit_bwv_derm,logit_vs_derm,logit_uncertainty_derm),
            (logit_diagnosis, logit_pn, logit_str, logit_pig, logit_rs, logit_dag, logit_bwv,logit_vs,logit_uncertainty),
            (x_clic_cos,x_derm_cos)] = net((clinic_image,derm_image))
        
        prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis_clic), dim = 1)
            
        prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn_clic), dim = 1)

        prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str_clic), dim = 1)

        prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig_clic), dim = 1)

        prediction5 = torch.argmax(torch.nn.Softmax(dim=1)(logit_rs_clic), dim = 1)

        prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag_clic), dim = 1)

        prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv_clic), dim = 1)

        prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs_clic), dim = 1)
        
        diagnosis_label_t = diagnosis_label.unsqueeze(0)
        pn_label_t = pn_label.unsqueeze(0)
        str_label_t = str_label.unsqueeze(0)
        pig_label_t = pig_label.unsqueeze(0)
        rs_label_t = rs_label.unsqueeze(0)
        dag_label_t = dag_label.unsqueeze(0)
        bwv_label_t = bwv_label.unsqueeze(0)
        vs_label_t = vs_label.unsqueeze(0)
        
        prediction1 = prediction1.unsqueeze(0)
        prediction2 = prediction2.unsqueeze(0)
        prediction3 = prediction3.unsqueeze(0)
        prediction4 = prediction4.unsqueeze(0)
        prediction5 = prediction5.unsqueeze(0)
        prediction6 = prediction6.unsqueeze(0)
        prediction7 = prediction7.unsqueeze(0)
        prediction8 = prediction8.unsqueeze(0)
        
        #concatenate tensor
        pn_label_t = np.array(pn_label_t.cpu(),dtype="int")
        str_label_t = np.array(str_label_t.cpu(),dtype="int")
        pig_label_t = np.array(pig_label_t.cpu(),dtype="int")
        rs_label_t = np.array(rs_label_t.cpu(),dtype="int")
        dag_label_t = np.array(dag_label_t.cpu(),dtype="int")
        bwv_label_t = np.array(bwv_label_t.cpu(),dtype="int")
        vs_label_t = np.array(vs_label_t.cpu(),dtype="int")
        diagnosis_label_t = np.array(diagnosis_label_t.cpu(),dtype="int")
        
        #print("prediction1.size:",prediction1.size())
        prediction1 = np.array(prediction1.cpu(),dtype="int")
        prediction2 = np.array(prediction2.cpu(),dtype="int")
        prediction3 = np.array(prediction3.cpu(),dtype="int")
        prediction4 = np.array(prediction4.cpu(),dtype="int")
        prediction5 = np.array(prediction5.cpu(),dtype="int")
        prediction6 = np.array(prediction6.cpu(),dtype="int")
        prediction7 = np.array(prediction7.cpu(),dtype="int")
        prediction8 = np.array(prediction8.cpu(),dtype="int")
        
        t = np.hstack((diagnosis_label_t.T,pn_label_t.T))
        t = np.hstack((t,str_label_t.T))
        t = np.hstack((t,pig_label_t.T))
        t = np.hstack((t,rs_label_t.T))
        t = np.hstack((t,dag_label_t.T))
        t = np.hstack((t,bwv_label_t.T))
        t = np.hstack((t,vs_label_t.T))
        t = torch.tensor(t)
        p = np.hstack((prediction1.T,prediction2.T))
        p = np.hstack((p,prediction3.T))
        p = np.hstack((p,prediction4.T))
        p = np.hstack((p,prediction5.T))
        p = np.hstack((p,prediction6.T))
        p = np.hstack((p,prediction7.T))
        p = np.hstack((p,prediction8.T))
        p = torch.tensor(p)
        #print("t:",t)
        #print("p:",p)
        certainty_b = (t == p).long()
        #print("certainty_b:",certainty_b)
        #print("certainty_b.size:",certainty_b.size())
        certainty_ones = torch.tensor(np.ones(certainty_b.size()))
        #print("certainty_ones:",certainty_ones)
        #print("certainty_ones.size:",certainty_ones.size())
        uncertainty_label_clic = []
        for i in range(certainty_ones.size(0)):
            mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
            if mean_squared_error_tmp >=0.6:
                mean_squared_error_tmp = 1
            else:
                mean_squared_error_tmp = 0
            uncertainty_label_clic.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
        uncertainty_label_clic = torch.tensor(uncertainty_label_clic)
        #print("uncertainty_label_clic:",uncertainty_label_clic)
        
        #uncertainty
        uncertainty_label_clic = uncertainty_label_clic.float().cuda()
    
        
        loss_clic = torch.div(
          net.criterion(logit_diagnosis_clic, diagnosis_label)
        + net.criterion(logit_pn_clic, pn_label)
        + net.criterion(logit_str_clic, str_label)
        + net.criterion(logit_pig_clic, pig_label)
        + net.criterion(logit_rs_clic, rs_label)
        + net.criterion(logit_dag_clic, dag_label)
        + net.criterion(logit_bwv_clic, bwv_label)
        + net.criterion(logit_vs_clic, vs_label), 8)
        #print("loss_clic:",loss_clic)
        
        #derm
        prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis_derm),dim = 1)
        
        prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn_derm),dim = 1)

        prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str_derm),dim = 1)

        prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig_derm), dim = 1)

        prediction5 =torch.argmax(torch.nn.Softmax(dim=1)(logit_rs_derm), dim = 1)

        prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag_derm), dim = 1)

        prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv_derm), dim = 1)

        prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs_clic), dim = 1)

        prediction1 = prediction1.unsqueeze(0)
        prediction2 = prediction2.unsqueeze(0)
        prediction3 = prediction3.unsqueeze(0)
        prediction4 = prediction4.unsqueeze(0)
        prediction5 = prediction5.unsqueeze(0)
        prediction6 = prediction6.unsqueeze(0)
        prediction7 = prediction7.unsqueeze(0)
        prediction8 = prediction8.unsqueeze(0)
        prediction1 = np.array(prediction1.cpu(),dtype="int")
        prediction2 = np.array(prediction2.cpu(),dtype="int")
        prediction3 = np.array(prediction3.cpu(),dtype="int")
        prediction4 = np.array(prediction4.cpu(),dtype="int")
        prediction5 = np.array(prediction5.cpu(),dtype="int")
        prediction6 = np.array(prediction6.cpu(),dtype="int")
        prediction7 = np.array(prediction7.cpu(),dtype="int")
        prediction8 = np.array(prediction8.cpu(),dtype="int")
        
        p = np.hstack((prediction1.T,prediction2.T))
        p = np.hstack((p,prediction3.T))
        p = np.hstack((p,prediction4.T))
        p = np.hstack((p,prediction5.T))
        p = np.hstack((p,prediction6.T))
        p = np.hstack((p,prediction7.T))
        p = np.hstack((p,prediction8.T))
        p = torch.tensor(p)
        
        certainty_b = (t == p).long()
        certainty_ones = torch.tensor(np.ones(certainty_b.size()))
        uncertainty_label_derm = []
        for i in range(certainty_ones.size(0)):
            mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
            if mean_squared_error_tmp >=0.6:
                mean_squared_error_tmp = 1
            else:
                mean_squared_error_tmp = 0
            uncertainty_label_derm.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
        uncertainty_label_derm = torch.tensor(uncertainty_label_derm)
        #uncertainty
        uncertainty_label_derm = uncertainty_label_derm.float().cuda()
        
        loss_derm = torch.div(
          net.criterion(logit_diagnosis_derm, diagnosis_label)
        + net.criterion(logit_pn_derm, pn_label)
        + net.criterion(logit_str_derm, str_label)
        + net.criterion(logit_pig_derm, pig_label)
        + net.criterion(logit_rs_derm, rs_label)
        + net.criterion(logit_dag_derm, dag_label)
        + net.criterion(logit_bwv_derm, bwv_label)
        + net.criterion(logit_vs_derm, vs_label), 8)
        
        #concat
        prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis),dim = 1)
        
        prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn),dim = 1)

        prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str),dim = 1)

        prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig), dim = 1)
    
        prediction5 = torch.argmax(torch.nn.Softmax(dim=1)(logit_rs),dim = 1)

        prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag),dim = 1)
        
        prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv),dim = 1)
        
        prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs),dim = 1)
        
        prediction1 = prediction1.unsqueeze(0)
        prediction2 = prediction2.unsqueeze(0)
        prediction3 = prediction3.unsqueeze(0)
        prediction4 = prediction4.unsqueeze(0)
        prediction5 = prediction5.unsqueeze(0)
        prediction6 = prediction6.unsqueeze(0)
        prediction7 = prediction7.unsqueeze(0)
        prediction8 = prediction8.unsqueeze(0)
        prediction1 = np.array(prediction1.cpu(),dtype="int")
        prediction2 = np.array(prediction2.cpu(),dtype="int")
        prediction3 = np.array(prediction3.cpu(),dtype="int")
        prediction4 = np.array(prediction4.cpu(),dtype="int")
        prediction5 = np.array(prediction5.cpu(),dtype="int")
        prediction6 = np.array(prediction6.cpu(),dtype="int")
        prediction7 = np.array(prediction7.cpu(),dtype="int")
        prediction8 = np.array(prediction8.cpu(),dtype="int")
        
        p = np.hstack((prediction1.T,prediction2.T))
        p = np.hstack((p,prediction3.T))
        p = np.hstack((p,prediction4.T))
        p = np.hstack((p,prediction5.T))
        p = np.hstack((p,prediction6.T))
        p = np.hstack((p,prediction7.T))
        p = np.hstack((p,prediction8.T))
        p = torch.tensor(p)
        
        certainty_b = (t == p).long()
        certainty_ones = torch.tensor(np.ones(certainty_b.size()))
        uncertainty_label = []
        for i in range(certainty_ones.size(0)):
            mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
            if mean_squared_error_tmp >=0.6:
                mean_squared_error_tmp = 1
            else:
                mean_squared_error_tmp = 0
            uncertainty_label.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
        uncertainty_label = torch.tensor(uncertainty_label)
        #uncertainty
        uncertainty_label = uncertainty_label.float().cuda()
        
        loss_concat = torch.div(
          net.criterion(logit_diagnosis, diagnosis_label)
        + net.criterion(logit_pn, pn_label)
        + net.criterion(logit_str, str_label)
        + net.criterion(logit_pig, pig_label)
        + net.criterion(logit_rs, rs_label)
        + net.criterion(logit_dag, dag_label)
        + net.criterion(logit_bwv, bwv_label)
        + net.criterion(logit_vs, vs_label), 8)
        
        x_clic_cos = x_clic_cos.float()
        x_derm_cos = x_derm_cos.float()
        criterion = nn.CosineEmbeddingLoss()
        Tar = np.ones((x_clic_cos.size(0)))
        Tar = torch.Tensor(Tar).cuda()
        loss_cos = criterion(x_clic_cos, x_derm_cos,Tar)
        loss = loss_concat*0.8 + loss_cos*0.2    #(0.1,0.2)
        #loss = loss_concat*0.9 + loss_cos*0.1
        #loss = loss_clic*0.5 + loss_derm*0.5
        
        #print("1:",net.metric(logit_diagnosis_fusion, diagnosis_label))
        dia_acc_all = torch.div(net.metric(logit_diagnosis,diagnosis_label),int(clinic_image.size(0)))
        sps_acc_all = torch.div(  net.metric(logit_pn,pn_label)
                                + net.metric(logit_str,str_label)
                                + net.metric(logit_pig,pig_label)
                                + net.metric(logit_rs,rs_label)
                                + net.metric(logit_dag,dag_label)
                                + net.metric(logit_bwv,bwv_label)
                                + net.metric(logit_vs,vs_label),int(7 * clinic_image.size(0)))
        loss.backward()
        opt.step()

        train_loss += loss.item()
        train_dia_acc += dia_acc_all.item()
        train_sps_acc += sps_acc_all.item()

    train_loss = train_loss / (index + 1) # Because the index start with the value 0f zero
    train_dia_acc = train_dia_acc / (index + 1)
    train_sps_acc = train_sps_acc / (index + 1)

    return train_loss,train_dia_acc,train_sps_acc

In [3]:
def validation(net,val_dataloader,model_name):
    net.set_mode('valid')
    val_loss = 0
    val_dia_acc = 0
    vaL_sps_acc = 0
    for index, (clinic_image, derm_image, meta_data, label) in enumerate(val_dataloader):

        clinic_image = clinic_image.cuda()
        derm_image   = derm_image.cuda()
        #meta_data    = meta_data.cuda()

        diagnosis_label = label[0].long().cuda()
        pn_label = label[1].long().cuda()
        str_label = label[2].long().cuda()
        pig_label = label[3].long().cuda()
        rs_label = label[4].long().cuda()
        dag_label = label[5].long().cuda()
        bwv_label = label[6].long().cuda()
        vs_label = label[7].long().cuda()
        

        with torch.no_grad():

            [   (logit_diagnosis_clic,logit_pn_clic,logit_str_clic,logit_pig_clic,logit_rs_clic,logit_dag_clic,logit_bwv_clic,logit_vs_clic,logit_uncertainty_clic),
            (logit_diagnosis_derm,logit_pn_derm,logit_str_derm,logit_pig_derm,logit_rs_derm,logit_dag_derm,logit_bwv_derm,logit_vs_derm,logit_uncertainty_derm),
            (logit_diagnosis, logit_pn, logit_str, logit_pig, logit_rs, logit_dag, logit_bwv,logit_vs,logit_uncertainty),
            (x_clic_cos,x_derm_cos)] = net((clinic_image,derm_image))
            
            prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis_clic), dim = 1)
            
            prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn_clic), dim = 1)

            prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str_clic), dim = 1)

            prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig_clic), dim = 1)

            prediction5 = torch.argmax(torch.nn.Softmax(dim=1)(logit_rs_clic), dim = 1)

            prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag_clic), dim = 1)

            prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv_clic), dim = 1)

            prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs_clic), dim = 1)
        
            diagnosis_label_t = diagnosis_label.unsqueeze(0)
            pn_label_t = pn_label.unsqueeze(0)
            str_label_t = str_label.unsqueeze(0)
            pig_label_t = pig_label.unsqueeze(0)
            rs_label_t = rs_label.unsqueeze(0)
            dag_label_t = dag_label.unsqueeze(0)
            bwv_label_t = bwv_label.unsqueeze(0)
            vs_label_t = vs_label.unsqueeze(0)
        
            prediction1 = prediction1.unsqueeze(0)
            prediction2 = prediction2.unsqueeze(0)
            prediction3 = prediction3.unsqueeze(0)
            prediction4 = prediction4.unsqueeze(0)
            prediction5 = prediction5.unsqueeze(0)
            prediction6 = prediction6.unsqueeze(0)
            prediction7 = prediction7.unsqueeze(0)
            prediction8 = prediction8.unsqueeze(0)
        
            #concatenate tensor
            pn_label_t = np.array(pn_label_t.cpu(),dtype="int")
            str_label_t = np.array(str_label_t.cpu(),dtype="int")
            pig_label_t = np.array(pig_label_t.cpu(),dtype="int")
            rs_label_t = np.array(rs_label_t.cpu(),dtype="int")
            dag_label_t = np.array(dag_label_t.cpu(),dtype="int")
            bwv_label_t = np.array(bwv_label_t.cpu(),dtype="int")
            vs_label_t = np.array(vs_label_t.cpu(),dtype="int")
            diagnosis_label_t = np.array(diagnosis_label_t.cpu(),dtype="int")
        
            prediction1 = np.array(prediction1.cpu(),dtype="int")
            prediction2 = np.array(prediction2.cpu(),dtype="int")
            prediction3 = np.array(prediction3.cpu(),dtype="int")
            prediction4 = np.array(prediction4.cpu(),dtype="int")
            prediction5 = np.array(prediction5.cpu(),dtype="int")
            prediction6 = np.array(prediction6.cpu(),dtype="int")
            prediction7 = np.array(prediction7.cpu(),dtype="int")
            prediction8 = np.array(prediction8.cpu(),dtype="int")
        
            t = np.hstack((diagnosis_label_t.T,pn_label_t.T))
            t = np.hstack((t,str_label_t.T))
            t = np.hstack((t,pig_label_t.T))
            t = np.hstack((t,rs_label_t.T))
            t = np.hstack((t,dag_label_t.T))
            t = np.hstack((t,bwv_label_t.T))
            t = np.hstack((t,vs_label_t.T))
            t = torch.tensor(t)
            p = np.hstack((prediction1.T,prediction2.T))
            p = np.hstack((p,prediction3.T))
            p = np.hstack((p,prediction4.T))
            p = np.hstack((p,prediction5.T))
            p = np.hstack((p,prediction6.T))
            p = np.hstack((p,prediction7.T))
            p = np.hstack((p,prediction8.T))
            p = torch.tensor(p)
            certainty_b = (t == p).long()
            certainty_ones = torch.tensor(np.ones(certainty_b.size()))
            uncertainty_label_clic = []
            for i in range(certainty_ones.size(0)):
                mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
                if mean_squared_error_tmp >=0.6:
                    mean_squared_error_tmp = 1
                else:
                    mean_squared_error_tmp = 0
                uncertainty_label_clic.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
            uncertainty_label_clic = torch.tensor(uncertainty_label_clic)
        
            #uncertainty
            uncertainty_label_clic = uncertainty_label_clic.float().cuda()    
        
            loss_clic = torch.div(
              net.criterion(logit_diagnosis_clic, diagnosis_label)
            + net.criterion(logit_pn_clic, pn_label)
            + net.criterion(logit_str_clic, str_label)
            + net.criterion(logit_pig_clic, pig_label)
            + net.criterion(logit_rs_clic, rs_label)
            + net.criterion(logit_dag_clic, dag_label)
            + net.criterion(logit_bwv_clic, bwv_label)
            + net.criterion(logit_vs_clic, vs_label), 8)
        
            #derm
            prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis_derm), dim = 1)
        
            prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn_derm), dim = 1)

            prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str_derm), dim = 1)

            prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig_derm), dim = 1)

            prediction5 = torch.argmax(torch.nn.Softmax(dim=1)(logit_rs_derm), dim = 1)

            prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag_derm), dim = 1)
    
            prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv_derm), dim = 1)

            prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs_clic), dim = 1)

            prediction1 = prediction1.unsqueeze(0)
            prediction2 = prediction2.unsqueeze(0)
            prediction3 = prediction3.unsqueeze(0)
            prediction4 = prediction4.unsqueeze(0)
            prediction5 = prediction5.unsqueeze(0)
            prediction6 = prediction6.unsqueeze(0)
            prediction7 = prediction7.unsqueeze(0)
            prediction8 = prediction8.unsqueeze(0)
            prediction1 = np.array(prediction1.cpu(),dtype="int")
            prediction2 = np.array(prediction2.cpu(),dtype="int")
            prediction3 = np.array(prediction3.cpu(),dtype="int")
            prediction4 = np.array(prediction4.cpu(),dtype="int")
            prediction5 = np.array(prediction5.cpu(),dtype="int")
            prediction6 = np.array(prediction6.cpu(),dtype="int")
            prediction7 = np.array(prediction7.cpu(),dtype="int")
            prediction8 = np.array(prediction8.cpu(),dtype="int")
        
            p = np.hstack((prediction1.T,prediction2.T))
            p = np.hstack((p,prediction3.T))
            p = np.hstack((p,prediction4.T))
            p = np.hstack((p,prediction5.T))
            p = np.hstack((p,prediction6.T))
            p = np.hstack((p,prediction7.T))
            p = np.hstack((p,prediction8.T))
            p = torch.tensor(p)
        
            certainty_b = (t == p).long()
            certainty_ones = torch.tensor(np.ones(certainty_b.size()))
            uncertainty_label_derm = []
            for i in range(certainty_ones.size(0)):
                mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
                if mean_squared_error_tmp >=0.6:
                    mean_squared_error_tmp = 1
                else:
                    mean_squared_error_tmp = 0
                uncertainty_label_derm.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
            uncertainty_label_derm = torch.tensor(uncertainty_label_derm)
            #uncertainty
            uncertainty_label_derm = uncertainty_label_derm.float().cuda()
        
            loss_derm = torch.div(
              net.criterion(logit_diagnosis_derm, diagnosis_label)
            + net.criterion(logit_pn_derm, pn_label)
            + net.criterion(logit_str_derm, str_label)
            + net.criterion(logit_pig_derm, pig_label)
            + net.criterion(logit_rs_derm, rs_label)
            + net.criterion(logit_dag_derm, dag_label)
            + net.criterion(logit_bwv_derm, bwv_label)
            + net.criterion(logit_vs_derm, vs_label), 8)

            #concat
            prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis), dim = 1)

            prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn), dim = 1)

            prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str), dim = 1)

            prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig), dim = 1)
    
            prediction5 = torch.argmax(torch.nn.Softmax(dim=1)(logit_rs), dim = 1)

            prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag), dim = 1)
        
            prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv), dim = 1)
        
            prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs), dim = 1)
        
            prediction1 = prediction1.unsqueeze(0)
            prediction2 = prediction2.unsqueeze(0)
            prediction3 = prediction3.unsqueeze(0)
            prediction4 = prediction4.unsqueeze(0)
            prediction5 = prediction5.unsqueeze(0)
            prediction6 = prediction6.unsqueeze(0)
            prediction7 = prediction7.unsqueeze(0)
            prediction8 = prediction8.unsqueeze(0)
            prediction1 = np.array(prediction1.cpu(),dtype="int")
            prediction2 = np.array(prediction2.cpu(),dtype="int")
            prediction3 = np.array(prediction3.cpu(),dtype="int")
            prediction4 = np.array(prediction4.cpu(),dtype="int")
            prediction5 = np.array(prediction5.cpu(),dtype="int")
            prediction6 = np.array(prediction6.cpu(),dtype="int")
            prediction7 = np.array(prediction7.cpu(),dtype="int")
            prediction8 = np.array(prediction8.cpu(),dtype="int")
        
            p = np.hstack((prediction1.T,prediction2.T))
            p = np.hstack((p,prediction3.T))
            p = np.hstack((p,prediction4.T))
            p = np.hstack((p,prediction5.T))
            p = np.hstack((p,prediction6.T))
            p = np.hstack((p,prediction7.T))
            p = np.hstack((p,prediction8.T))
            p = torch.tensor(p)
        
            certainty_b = (t == p).long()
            certainty_ones = torch.tensor(np.ones(certainty_b.size()))
            uncertainty_label = []
            for i in range(certainty_ones.size(0)):
                mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
                if mean_squared_error_tmp >=0.6:
                    mean_squared_error_tmp = 1
                else:
                    mean_squared_error_tmp = 0
                uncertainty_label.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
            uncertainty_label = torch.tensor(uncertainty_label)
            #uncertainty
            uncertainty_label = uncertainty_label.float().cuda()
        
            loss_concat = torch.div(
              net.criterion(logit_diagnosis, diagnosis_label)
            + net.criterion(logit_pn, pn_label)
            + net.criterion(logit_str, str_label)
            + net.criterion(logit_pig, pig_label)
            + net.criterion(logit_rs, rs_label)
            + net.criterion(logit_dag, dag_label)
            + net.criterion(logit_bwv, bwv_label)
            + net.criterion(logit_vs, vs_label), 8)
        
            x_clic_cos = x_clic_cos.float()
            x_derm_cos = x_derm_cos.float()
            criterion = nn.CosineEmbeddingLoss()
            Tar = np.ones((x_clic_cos.size(0)))
            Tar = torch.Tensor(Tar).cuda()
            loss_cos = criterion(x_clic_cos, x_derm_cos,Tar)
            loss = loss_concat*0.8 + loss_cos*0.2    #(0.1,0.2)
            #loss = loss_concat*0.9 + loss_cos*0.1
            #loss = loss_clic*0.5 + loss_derm*0.5
            
            dia_acc_all = torch.div(net.metric(logit_diagnosis,diagnosis_label),int(clinic_image.size(0)))
            sps_acc_all = torch.div(net.metric(logit_pn,pn_label)
                                + net.metric(logit_str,str_label)
                                + net.metric(logit_pig,pig_label)
                                + net.metric(logit_rs,rs_label)
                                + net.metric(logit_dag,dag_label)
                                + net.metric(logit_bwv,bwv_label)
                                + net.metric(logit_vs,vs_label),int(7 * clinic_image.size(0)))
            
                                    
        val_loss += loss.item()
        val_dia_acc += dia_acc_all.item()
        vaL_sps_acc += sps_acc_all.item()

    val_loss = val_loss / (index + 1)
    val_dia_acc = val_dia_acc / (index + 1)
    vaL_sps_acc = vaL_sps_acc / (index + 1)

    return val_loss,val_dia_acc,vaL_sps_acc

In [4]:
def run_train(model_name,mode,i):
    log.write('** start Resnet_with_cos-distance(without uncertainty) training here! **\n')
    #best_acc = 0
    es = 0
    patience = 50
    best_mean_acc = 0 
    best_loss = 300
    
    for epoch in range(epochs):
        swa_lr = cosine_learning_schedule[epoch]
        adjust_learning_rate(opt, swa_lr)

        # train_mode
        train_loss,train_dia_acc,train_sps_acc=train(net, train_dataloader,model_name)
        log.write('Round: {}, epoch: {}, Train Loss: {:.4f}, Train Dia Acc: {:.4f}, Train SPS Acc: {:.4f}\n'.format(i, epoch, train_loss,
                                                                                                         train_dia_acc,
                                                                                                         train_sps_acc))

        # validation mode
        val_loss,val_dia_acc,val_sps_acc = validation(net, val_dataloader,model_name)
        
        val_acc = (val_dia_acc + val_sps_acc) / 2
        val_mean_acc = (val_dia_acc*1 + val_sps_acc*7)/8
        
        log.write('Round: {}, epoch: {}, Valid Loss: {:.4f}, Valid Dia Acc: {:.4f}, Valid SPS Acc: {:.4f}\n'.format(i, epoch, val_loss,
                                                                                                         val_dia_acc,
                                                                                                         val_sps_acc))

     
        if val_mean_acc > best_mean_acc:
            es = 0
            best_mean_acc = val_mean_acc
            torch.save(net.state_dict(), out_dir + '/checkpoint/{}_model.pth'.format('best_mean_acc'))
            log.write('Current Best Mean Acc is {}'.format(best_mean_acc))
        #  else:
        #      es += 1
        #      print("Counter {} of {}".format(es,patience))
          
        #      if es > patience:
        #          print("Early stopping with best_mean_acc: {:.4f}".format(best_mean_acc), "and val_mean_acc for this epoch: {:.4f}".format(val_mean_acc))
        #          break
  
  
        if epoch > (epochs - swa_epoch) and epoch % 1 == 0:
            opt.update_swa()
            log.write('SWA Epoch: {}'.format(epoch))

    torch.save(net.state_dict(), out_dir+'/swa_{}_resnet50_model.pth'.format(mode))

In [5]:
if __name__ == '__main__':
    # Hyperparameters
    mode = 'multimodal'
    tech = "cos-loss and concat-without uncertainty"
    model_name = str('Resnet')+str("_")+str(tech)
    shape = (229, 229)
    batch_size = 5
    num_workers = 0
    data_mode = 'Normal'
    deterministic = True
    if deterministic:
        if data_mode == 'Normal':
            random_seeds = 170
        elif data_mode == 'self_evaluated':
            random_seeds = 183
    rounds = 2
    lr = 4.9e-5###
    epochs = 250
    swa_epoch = 150    

    train_dataloader, val_dataloader = generate_dataloader(shape, batch_size, num_workers, data_mode,"concat")
    
    for i in range(rounds):
        i =i + 3
        if deterministic:
            set_seed(random_seeds + i)
      # create logger
        print(random_seeds+i)
        log, out_dir = CreateLogger(mode, model_name,i,data_mode)
        #network 
        net = Resnet_with_uncertainty(class_list).cuda()
        #net = FusionNet(class_list).to(device)
      # create optimizer
        optimizer = optim.Adam(net.parameters(), lr=lr,weight_decay=5e-5)   #weight_decay   #5e-5
        opt = SWA(optimizer)  
      # create learning schedule
        cosine_learning_schedule = create_cosine_learning_schedule(epochs, lr)
        run_train(model_name,mode,i)

173

--- [START IDENTIFIER] ----------------------------------------------------------------


	out_dir      = ./multimodal_Resnet_cos-loss and concat-without uncertainty_Normal_weight_file/3/



	<additional comments> ...  

	  - multimodal Resnet_cos-loss and concat-without uncertainty  

	  - simple augmentation 



** start Resnet_with_cos-distance(without uncertainty) training here! **

Round: 3, epoch: 0, Train Loss: 1.0338, Train Dia Acc: 0.1751, Train SPS Acc: 0.3660

Round: 3, epoch: 0, Valid Loss: 0.9044, Valid Dia Acc: 0.2878, Valid SPS Acc: 0.3810

Current Best Mean Acc is 0.36930895251471824
Round: 3, epoch: 1, Train Loss: 1.0067, Train Dia Acc: 0.1807, Train SPS Acc: 0.3842

Round: 3, epoch: 1, Valid Loss: 0.8991, Valid Dia Acc: 0.2667, Valid SPS Acc: 0.4044

Current Best Mean Acc is 0.3871951301741164
Round: 3, epoch: 2, Train Loss: 0.9938, Train Dia Acc: 0.1976, Train SPS Acc: 0.4063

Round: 3, epoch: 2, Valid Loss: 0.8865, Valid Dia Acc: 0.3366, Valid SPS Acc: 0.4476



Round: 3, epoch: 40, Valid Loss: 0.6054, Valid Dia Acc: 0.6455, Valid SPS Acc: 0.7138

Current Best Mean Acc is 0.7052845692761788
Round: 3, epoch: 41, Train Loss: 0.5876, Train Dia Acc: 0.6659, Train SPS Acc: 0.7089

Round: 3, epoch: 41, Valid Loss: 0.6087, Valid Dia Acc: 0.6407, Valid SPS Acc: 0.7073

Round: 3, epoch: 42, Train Loss: 0.5840, Train Dia Acc: 0.7052, Train SPS Acc: 0.7087

Round: 3, epoch: 42, Valid Loss: 0.6002, Valid Dia Acc: 0.6488, Valid SPS Acc: 0.7152

Current Best Mean Acc is 0.7069105856211447
Round: 3, epoch: 43, Train Loss: 0.5807, Train Dia Acc: 0.6779, Train SPS Acc: 0.7025

Round: 3, epoch: 43, Valid Loss: 0.6014, Valid Dia Acc: 0.6602, Valid SPS Acc: 0.7178

Current Best Mean Acc is 0.7105691218067234
Round: 3, epoch: 44, Train Loss: 0.5907, Train Dia Acc: 0.6586, Train SPS Acc: 0.7060

Round: 3, epoch: 44, Valid Loss: 0.5834, Valid Dia Acc: 0.6732, Valid SPS Acc: 0.7243

Current Best Mean Acc is 0.7178861940870198
Round: 3, epoch: 45, Train Loss: 0.5832, 

Round: 3, epoch: 85, Train Loss: 0.4842, Train Dia Acc: 0.7229, Train SPS Acc: 0.7719

Round: 3, epoch: 85, Valid Loss: 0.5232, Valid Dia Acc: 0.6439, Valid SPS Acc: 0.7426

Round: 3, epoch: 86, Train Loss: 0.4573, Train Dia Acc: 0.7622, Train SPS Acc: 0.7816

Round: 3, epoch: 86, Valid Loss: 0.5218, Valid Dia Acc: 0.6634, Valid SPS Acc: 0.7398

Round: 3, epoch: 87, Train Loss: 0.4741, Train Dia Acc: 0.7221, Train SPS Acc: 0.7736

Round: 3, epoch: 87, Valid Loss: 0.5333, Valid Dia Acc: 0.6992, Valid SPS Acc: 0.7494

Round: 3, epoch: 88, Train Loss: 0.4704, Train Dia Acc: 0.7349, Train SPS Acc: 0.7775

Round: 3, epoch: 88, Valid Loss: 0.5255, Valid Dia Acc: 0.6748, Valid SPS Acc: 0.7361

Round: 3, epoch: 89, Train Loss: 0.4516, Train Dia Acc: 0.7791, Train SPS Acc: 0.7871

Round: 3, epoch: 89, Valid Loss: 0.5202, Valid Dia Acc: 0.6829, Valid SPS Acc: 0.7364

Round: 3, epoch: 90, Train Loss: 0.4795, Train Dia Acc: 0.7309, Train SPS Acc: 0.7702

Round: 3, epoch: 90, Valid Loss: 0.5206, Va

Round: 3, epoch: 129, Valid Loss: 0.5224, Valid Dia Acc: 0.6748, Valid SPS Acc: 0.7459

SWA Epoch: 129
Round: 3, epoch: 130, Train Loss: 0.4232, Train Dia Acc: 0.7622, Train SPS Acc: 0.7985

Round: 3, epoch: 130, Valid Loss: 0.5291, Valid Dia Acc: 0.6748, Valid SPS Acc: 0.7361

SWA Epoch: 130
Round: 3, epoch: 131, Train Loss: 0.4294, Train Dia Acc: 0.7695, Train SPS Acc: 0.8044

Round: 3, epoch: 131, Valid Loss: 0.5271, Valid Dia Acc: 0.6325, Valid SPS Acc: 0.7373

SWA Epoch: 131
Round: 3, epoch: 132, Train Loss: 0.4184, Train Dia Acc: 0.7831, Train SPS Acc: 0.8124

Round: 3, epoch: 132, Valid Loss: 0.5271, Valid Dia Acc: 0.6618, Valid SPS Acc: 0.7382

SWA Epoch: 132
Round: 3, epoch: 133, Train Loss: 0.4164, Train Dia Acc: 0.7518, Train SPS Acc: 0.8135

Round: 3, epoch: 133, Valid Loss: 0.5161, Valid Dia Acc: 0.6748, Valid SPS Acc: 0.7415

SWA Epoch: 133
Round: 3, epoch: 134, Train Loss: 0.4395, Train Dia Acc: 0.7534, Train SPS Acc: 0.8007

Round: 3, epoch: 134, Valid Loss: 0.5224, Val

Round: 3, epoch: 172, Valid Loss: 0.5035, Valid Dia Acc: 0.7073, Valid SPS Acc: 0.7577

Current Best Mean Acc is 0.7514227782716838
SWA Epoch: 172
Round: 3, epoch: 173, Train Loss: 0.3999, Train Dia Acc: 0.7711, Train SPS Acc: 0.8204

Round: 3, epoch: 173, Valid Loss: 0.5024, Valid Dia Acc: 0.6943, Valid SPS Acc: 0.7556

SWA Epoch: 173
Round: 3, epoch: 174, Train Loss: 0.3947, Train Dia Acc: 0.7831, Train SPS Acc: 0.8265

Round: 3, epoch: 174, Valid Loss: 0.5120, Valid Dia Acc: 0.6423, Valid SPS Acc: 0.7528

SWA Epoch: 174
Round: 3, epoch: 175, Train Loss: 0.3821, Train Dia Acc: 0.7863, Train SPS Acc: 0.8355

Round: 3, epoch: 175, Valid Loss: 0.5041, Valid Dia Acc: 0.6894, Valid SPS Acc: 0.7524

SWA Epoch: 175
Round: 3, epoch: 176, Train Loss: 0.3984, Train Dia Acc: 0.7767, Train SPS Acc: 0.8190

Round: 3, epoch: 176, Valid Loss: 0.5024, Valid Dia Acc: 0.7171, Valid SPS Acc: 0.7517

SWA Epoch: 176
Round: 3, epoch: 177, Train Loss: 0.3805, Train Dia Acc: 0.8032, Train SPS Acc: 0.8288

R

...

Round: 4, epoch: 49, Valid Loss: 0.5837, Valid Dia Acc: 0.6667, Valid SPS Acc: 0.7236

Current Best Mean Acc is 0.7164634306710668
Round: 4, epoch: 50, Train Loss: 0.5481, Train Dia Acc: 0.7068, Train SPS Acc: 0.7236

Round: 4, epoch: 50, Valid Loss: 0.6051, Valid Dia Acc: 0.6407, Valid SPS Acc: 0.7089

Round: 4, epoch: 51, Train Loss: 0.5461, Train Dia Acc: 0.7157, Train SPS Acc: 0.7344

Round: 4, epoch: 51, Valid Loss: 0.5799, Valid Dia Acc: 0.6439, Valid SPS Acc: 0.7185

Round: 4, epoch: 52, Train Loss: 0.5139, Train Dia Acc: 0.7092, Train SPS Acc: 0.7526

Round: 4, epoch: 52, Valid Loss: 0.5896, Valid Dia Acc: 0.6602, Valid SPS Acc: 0.7187

Round: 4, epoch: 53, Train Loss: 0.5428, Train Dia Acc: 0.6956, Train SPS Acc: 0.7315

Round: 4, epoch: 53, Valid Loss: 0.5795, Valid Dia Acc: 0.6585, Valid SPS Acc: 0.7192

Round: 4, epoch: 54, Train Loss: 0.5443, Train Dia Acc: 0.7036, Train SPS Acc: 0.7359

Round: 4, epoch: 54, Valid Loss: 0.5851, Valid Dia Acc: 0.6439, Valid SPS Acc: 0.7075


Round: 4, epoch: 95, Train Loss: 0.4621, Train Dia Acc: 0.7719, Train SPS Acc: 0.7791

Round: 4, epoch: 95, Valid Loss: 0.5351, Valid Dia Acc: 0.6439, Valid SPS Acc: 0.7345

Round: 4, epoch: 96, Train Loss: 0.4559, Train Dia Acc: 0.7438, Train SPS Acc: 0.7788

Round: 4, epoch: 96, Valid Loss: 0.5371, Valid Dia Acc: 0.6504, Valid SPS Acc: 0.7398

Round: 4, epoch: 97, Train Loss: 0.4652, Train Dia Acc: 0.7430, Train SPS Acc: 0.7801

Round: 4, epoch: 97, Valid Loss: 0.5385, Valid Dia Acc: 0.6358, Valid SPS Acc: 0.7424

Round: 4, epoch: 98, Train Loss: 0.4494, Train Dia Acc: 0.7165, Train SPS Acc: 0.7940

Round: 4, epoch: 98, Valid Loss: 0.5395, Valid Dia Acc: 0.6797, Valid SPS Acc: 0.7512

Current Best Mean Acc is 0.7422764374897247
Round: 4, epoch: 99, Train Loss: 0.4508, Train Dia Acc: 0.7542, Train SPS Acc: 0.7931

Round: 4, epoch: 99, Valid Loss: 0.5284, Valid Dia Acc: 0.6423, Valid SPS Acc: 0.7452

Round: 4, epoch: 100, Train Loss: 0.4534, Train Dia Acc: 0.7406, Train SPS Acc: 0.7906

Round: 4, epoch: 138, Train Loss: 0.4016, Train Dia Acc: 0.7671, Train SPS Acc: 0.8202

Round: 4, epoch: 138, Valid Loss: 0.5205, Valid Dia Acc: 0.6829, Valid SPS Acc: 0.7429

SWA Epoch: 138
Round: 4, epoch: 139, Train Loss: 0.4107, Train Dia Acc: 0.7743, Train SPS Acc: 0.8132

Round: 4, epoch: 139, Valid Loss: 0.5168, Valid Dia Acc: 0.6504, Valid SPS Acc: 0.7391

SWA Epoch: 139
Round: 4, epoch: 140, Train Loss: 0.4108, Train Dia Acc: 0.7839, Train SPS Acc: 0.8171

Round: 4, epoch: 140, Valid Loss: 0.5242, Valid Dia Acc: 0.6813, Valid SPS Acc: 0.7394

SWA Epoch: 140
Round: 4, epoch: 141, Train Loss: 0.4162, Train Dia Acc: 0.7430, Train SPS Acc: 0.8116

Round: 4, epoch: 141, Valid Loss: 0.5218, Valid Dia Acc: 0.6911, Valid SPS Acc: 0.7384

SWA Epoch: 141
Round: 4, epoch: 142, Train Loss: 0.4218, Train Dia Acc: 0.7510, Train SPS Acc: 0.8132

Round: 4, epoch: 142, Valid Loss: 0.5256, Valid Dia Acc: 0.6602, Valid SPS Acc: 0.7343

SWA Epoch: 142
Round: 4, epoch: 143, Train Loss: 0.3984, Tra

Round: 4, epoch: 181, Train Loss: 0.3836, Train Dia Acc: 0.7574, Train SPS Acc: 0.8402

Round: 4, epoch: 181, Valid Loss: 0.5224, Valid Dia Acc: 0.6520, Valid SPS Acc: 0.7431

SWA Epoch: 181
Round: 4, epoch: 182, Train Loss: 0.3927, Train Dia Acc: 0.7430, Train SPS Acc: 0.8286

Round: 4, epoch: 182, Valid Loss: 0.5213, Valid Dia Acc: 0.6602, Valid SPS Acc: 0.7375

SWA Epoch: 182
Round: 4, epoch: 183, Train Loss: 0.3835, Train Dia Acc: 0.7574, Train SPS Acc: 0.8350

Round: 4, epoch: 183, Valid Loss: 0.5181, Valid Dia Acc: 0.7041, Valid SPS Acc: 0.7368

SWA Epoch: 183
Round: 4, epoch: 184, Train Loss: 0.3855, Train Dia Acc: 0.7663, Train SPS Acc: 0.8295

Round: 4, epoch: 184, Valid Loss: 0.5235, Valid Dia Acc: 0.6780, Valid SPS Acc: 0.7505

SWA Epoch: 184
Round: 4, epoch: 185, Train Loss: 0.3843, Train Dia Acc: 0.7952, Train SPS Acc: 0.8337

Round: 4, epoch: 185, Valid Loss: 0.5140, Valid Dia Acc: 0.6829, Valid SPS Acc: 0.7494

SWA Epoch: 185
Round: 4, epoch: 186, Train Loss: 0.3824, Tra

Round: 4, epoch: 224, Train Loss: 0.3610, Train Dia Acc: 0.7928, Train SPS Acc: 0.8487

Round: 4, epoch: 224, Valid Loss: 0.5213, Valid Dia Acc: 0.6520, Valid SPS Acc: 0.7398

SWA Epoch: 224
Round: 4, epoch: 225, Train Loss: 0.3712, Train Dia Acc: 0.7807, Train SPS Acc: 0.8327

Round: 4, epoch: 225, Valid Loss: 0.5188, Valid Dia Acc: 0.6862, Valid SPS Acc: 0.7443

SWA Epoch: 225
Round: 4, epoch: 226, Train Loss: 0.3734, Train Dia Acc: 0.7735, Train SPS Acc: 0.8386

Round: 4, epoch: 226, Valid Loss: 0.5142, Valid Dia Acc: 0.7024, Valid SPS Acc: 0.7422

SWA Epoch: 226
Round: 4, epoch: 227, Train Loss: 0.3736, Train Dia Acc: 0.7454, Train SPS Acc: 0.8368

Round: 4, epoch: 227, Valid Loss: 0.5175, Valid Dia Acc: 0.6585, Valid SPS Acc: 0.7384

SWA Epoch: 227
Round: 4, epoch: 228, Train Loss: 0.3817, Train Dia Acc: 0.7711, Train SPS Acc: 0.8361

Round: 4, epoch: 228, Valid Loss: 0.5133, Valid Dia Acc: 0.6715, Valid SPS Acc: 0.7454

SWA Epoch: 228
Round: 4, epoch: 229, Train Loss: 0.3791, Tra